In [1]:
import warnings
from typing import List

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from autumn.core.project import get_project
from autumn.settings import Region, Models
from autumn.core import inputs
from autumn.core.utils.display import pretty_print
from autumn.core.inputs.tb_camau import queries
from autumn.core.inputs import get_death_rates_by_agegroup
import pathlib

In [2]:
warnings.filterwarnings("ignore")
pd.options.plotting.backend = "plotly"

In [3]:
region = Region.CAMAU
model = Models.TBD2
p = get_project(model, region, reload=True)
baseline_params = p.param_set.baseline
plots = p.plots
iso3 = baseline_params['country']['iso3']
source_iso3 = baseline_params['age_mixing']['source_iso3']
age_adjust = baseline_params['age_mixing']['age_adjust']
age_string_map = {
    0: "0-4",
    5: "5-14",
    15: "15-34",
    35: "35-49",
    50: "50+",
}

## Population

In [4]:
modelled_age_groups = baseline_params["age_breakpoints"]
print(f"Modelled age groups are {modelled_age_groups}")

Modelled age groups are [0, 5, 15, 35, 50]


In [5]:
csv_path = pathlib.Path('camau.csv')

In [6]:
pop_df = pd.read_csv(csv_path)
pop_df = pop_df.set_index(['year'])
pop_df['population'].plot()

### Birth rate

In [7]:
birth_rates, years = inputs.get_crude_birth_rate("VNM")
birth_rates = [b / 1000.0 for b in birth_rates]  # Birth rates are provided / 1000 population
birth_rates_series = pd.Series(birth_rates, index=years)
br_fig= px.line(birth_rates_series)
br_fig.update_traces(mode='markers+lines')
br_fig.update_layout(
    title="Crude birth rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude birth rate",
    showlegend=False,
)

### Death rate

In [8]:
# Currently used in model:
death_rates_from_db = get_death_rates_by_agegroup(modelled_age_groups, "VNM")
death_rates_df = pd.DataFrame(death_rates_from_db[0], index=death_rates_from_db[1])
life_exp_at_age_df = 1.0 / death_rates_df
death_rates_df.plot()
life_exp_at_age_df.plot(log_y=True)

In [9]:
# Query and visualize the crude birth rate from database
years, death_rates = queries.get_camau_death_rate()
death_rates = [b / 1000.0 for b in death_rates]  # Birth rates are provided / 1000 population
death_rates_series = pd.Series(death_rates, index=years)
dr_fig= px.line(death_rates_series)
dr_fig.update_traces(mode='markers+lines')
dr_fig.update_layout(
    title="Crude death rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude death rate",
    showlegend=False,
)

### Mixing matrix

In [10]:
print(f"Target region: {region.upper()} ({iso3})")
print(f"Proxy country: {source_iso3}")
print("Always age-adjusted to target population")

Target region: CAMAU (VNM)
Proxy country: VNM
Always age-adjusted to target population


In [11]:
update_params = {
    'cdr_adjustment': 0.6014069880079691,
    'contact_rate': 0.0040506548106069165,
    'gender.adjustments.infection.male': 2.8723724227427603,
    'infect_death_rate_dict.smear_negative': 0.024849738227234807,
    'infect_death_rate_dict.smear_positive': 0.43186926176080265,
    'progression_multiplier': 1.990543692342679,
    'rr_infection_latent': 0.4806436742741195,
    'rr_infection_recovered': 0.48873928709530823,
    'self_recovery_rate_dict.smear_positive': 0.1803308001104804,
    'self_recovery_rate_dict.smear_negative': 0.2504239970722082,
    'start_population_size': 224776.83727076856
}
params = baseline_params.update(update_params,calibration_format=True)
model_0  =  p.build_model(params.to_dict())
model_0.run(params.to_dict())
derived_df_0 = model_0.get_derived_outputs_df()

In [12]:
model_0._stratifications

[Stratification: age, Stratification: gender, Stratification: organ]

In [13]:
model_0._mixing_matrices

[array([[ 398.43289672,  261.82020387,  643.68286218,  401.62199159,
          356.13449939],
        [ 165.78966683,  881.63067677,  532.84120554,  550.75979227,
          285.62836724],
        [ 231.75164317,  311.38983781,  915.52884268,  673.30894113,
          664.14577066],
        [ 141.94492435,  310.88835505,  786.13676958, 1134.31076003,
          938.03403291],
        [  67.30073632,  170.46333134,  647.30153978, 1018.81243422,
         1763.57657715]])]

In [14]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [15]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="LTBI percentage"
)
latent_plot.show()

In [16]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notificationsXgender_male",
)
# notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
# notif_2.update_traces(marker=dict(color="red"))
# notif_plot = go.Figure(
#     data=notif_1.data + notif_2.data,
# )
# notif_plot.update_layout(
#     title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
# )
notif_1.show()

In [17]:
inci_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="incidence",
)
inci_plot.update_layout(
    title="", title_x=0.5, xaxis_title="Year", yaxis_title="Incidence per 100,000 population"
)
inci_plot.show()

In [18]:
prev_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="prevalence_infectious",
)
prev_plot.update_layout(
    title="", title_x=0.5, xaxis_title="Year", yaxis_title="Prevalance per 100,000 population"
)
prev_plot.show()

In [19]:
output = model_0.get_outputs_df()

In [20]:
derived_df_0

,cumulative_deaths,cumulative_diseased,incidence,incidence_early,incidence_late,notifications,notificationsXgender_female,notificationsXgender_male,percentage_latent,prevalence_infectious,total_population,total_populationXgender_female,total_populationXgender_male
1900.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.781829,0.383096,0.398733,0.000000,0.444886,2.247768e+05,110140.650263,114636.187008
1900.1,0.000000,0.000000,0.367858,0.825908,0.000799,0.765944,0.375081,0.390863,0.000279,0.429798,2.247353e+05,110129.227354,114606.097480
1900.2,0.000000,0.000000,0.939899,2.108267,0.003680,0.740061,0.361564,0.378497,0.000538,0.420494,2.246993e+05,110120.432247,114578.833340
1900.3,0.000000,0.000000,1.252679,2.805608,0.008768,0.723705,0.352083,0.371621,0.000782,0.414695,2.246686e+05,110114.237721,114554.363817
1900.4,0.000000,0.000000,1.440055,3.219557,0.015429,0.712540,0.344712,0.367828,0.001018,0.410241,2.246433e+05,110110.616939,114532.658598
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019.6,201130.119259,697319.679710,1735.679340,8963.460790,9821.262672,10223.457866,3840.902981,6382.554885,37.943496,217.645773,1.082269e+06,543746.061268,538523.170774
2019.7,201616.252636,699198.540629,1735.781314,8975.357885,9813.251303,10216.073894,3837.151738,6378.922156,37.912866,217.448165,1.082430e+06,543824.199842,538605.311237
2019.8,202102.096635,701077.846464,1735.937718,8987.726204,9805.332149,10209.599239,3833.672692,6375.926547,37.882518,217.268636,1.082588e+06,543901.586933,538686.696824
2019.9,202587.689848,702957.347535,1735.865889,8997.508269,9797.502441,10203.864944,3830.402378,6373.462566,37.852262,217.102697,1.082746e+06,543978.224683,538767.327991


In [24]:
res = model_0._get_step_test()

TypeError: 'NoneType' object is not subscriptable

In [23]:
res

<bound method CompartmentalModel._get_step_test of <summer2.model.CompartmentalModel object at 0x000002667FED3610>>